# Bereinigung des Gerichtsurteils-Presseerklärungs-Datensatzes

Dieses Notebook implementiert verschiedene Ansätze zur Bereinigung des Datensatzes mit Gerichtsurteilen und zugehörigen Pressemitteilungen. Das Ziel ist die Identifikation und Entfernung von Pressemitteilungen, die keinen direkten Bezug zu einem ergangenen Urteil haben (z.B. Ankündigungen zukünftiger Verhandlungen oder allgemeine Mitteilungen).


## Überblick

Wir implementieren und evaluieren folgende Ansätze:

1. **Regelbasierte Filter**: 
   - Keywords und Muster zur Erkennung von Ankündigungen
   - Regex-Patterns für typische Formulierungen
   - Strukturelle Merkmale der Texte

2. **Semantische Ähnlichkeit**:
   - Berechnung von Text-Embeddings
   - Kosinus-Ähnlichkeit zwischen Urteil und Pressemitteilung
   - Identifikation inhaltlich abweichender Mitteilungen

3. **Überwachtes Machine Learning**:
   - Trainieren eines Klassifikators auf Basis der vorherigen Methoden
   - Logistische Regression mit TF-IDF Features
   - Analyse der wichtigsten Merkmale

4. **Unüberwachtes Clustering**:
   - Gruppierung ähnlicher Pressemitteilungen
   - Topic Modeling zur Mustererkennung
   - Identifikation von Ankündigungs-Clustern

Am Ende vergleichen wir die Methoden und kombinieren sie zu einem robusten Gesamtansatz.

## Erweiterte Bereinigungsansätze

In diesem Abschnitt implementieren wir weitere Ansätze zur Bereinigung des Datensatzes, die in `ansatz_bereinigung.md` beschrieben wurden:

1. Erweiterter regelbasierter Ansatz
2. Semantische Ähnlichkeit mit Embeddings
3. Überwachtes Machine Learning (Klassifikation)
4. Unüberwachte Verfahren (Clustering/Topic Modeling)



## 1. Vorbereitung für erweiterte Bereinigungsansätze

In [2]:
import re
import spacy
import numpy as np
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# GPU-Beschleunigung importieren
try:
    import cudf
    import cuml
    import cupy as cp
    from cuml.feature_extraction.text import TfidfVectorizer as cuTfidfVectorizer
    from cuml.decomposition import PCA as cuPCA
    from cuml.cluster import KMeans as cuKMeans
    from cuml.linear_model import LogisticRegression as cuLogisticRegression
    from cuml.metrics import pairwise_distances
    from cuml.preprocessing import normalize
    USE_GPU = True
    print("GPU-Beschleunigung ist aktiviert (RAPIDS-Bibliotheken geladen)")
except ImportError:
    USE_GPU = False
    print("GPU-Beschleunigung ist nicht verfügbar, verwende CPU-Version")

# Für alle Methoden laden wir die Daten neu
try:
    df = pd.read_csv('data/german_courts.csv')
    print(f"Daten geladen: {len(df)} Einträge")
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")
    # Fallback auf den bereits geladenen DataFrame, falls vorhanden

# Sprachdaten vorbereiten
try:
    nlp = spacy.load("de_core_news_sm")
    print("SpaCy deutsches Modell geladen")
except:
    print("SpaCy deutsches Modell wird installiert...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "de_core_news_sm"])
    nlp = spacy.load("de_core_news_sm")
    print("SpaCy deutsches Modell geladen")

# Kleine Hilfsfunktion zum Preprocessen von Text
def preprocess_text(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    # Nur Substantive, Verben, Adjektive und Adverbien behalten
    tokens = [token.lemma_.lower() for token in doc 
              if not token.is_stop and not token.is_punct 
              and token.is_alpha and len(token.text) > 2 
              and token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']]
    return " ".join(tokens)

# Sample für schnelleres Testen erstellen (optional)
sample_size = 1000  # Anzahl der zufällig ausgewählten Datensätze
sample_df = df.sample(min(sample_size, len(df)), random_state=42)

Daten geladen: 6592 Einträge
SpaCy deutsches Modell geladen


## 2. Regelbasierte Filter

Der erste Bereinigungsansatz basiert auf expliziten Regeln, die typische Muster in Ankündigungen und irrelevanten Pressemitteilungen erkennen:

- **Keywords**: Typische Wörter für Ankündigungen (z.B. "Terminankündigung", "wird verhandelt")
- **Zeitliche Marker**: Formulierungen, die auf zukünftige Ereignisse hinweisen
- **Dokumentstruktur**: Analyse von Überschriften und Textanfängen
- **Kombinierte Regeln**: Gewichtung verschiedener Indikatoren

Die Funktion `is_announcement_rule_based()` implementiert diese Logik und klassifiziert jeden Eintrag als Ankündigung oder relevante Pressemitteilung.

In [3]:
### 1. Erweiterter regelbasierter Ansatz (Keywords und Muster)

# Wir erweitern den bereits vorhandenen regelbasierten Ansatz mit detaillierteren Regex-Mustern 
# und umfangreicheren Wortlisten, um Ankündigungen und allgemeine Mitteilungen zu identifizieren

def is_announcement_rule_based(row):
    """Erkennt Ankündigungen und nicht urteilsbezogene Mitteilungen anhand von Keywords und Mustern"""
    if pd.isna(row['summary']):
        return False
    
    summary = str(row['summary']).lower()
    
    # 1. Keywords, die auf Ankündigungen hindeuten
    future_indicators = [
        'ankündigung', 'terminankündigung', 'terminhinweis', 'hinweis auf termin',
        'wird verhandelt', 'wird verhandeln', 'wird.*stattfinden', 'findet statt',
        'einladung', 'pressetermin', 'pressekonferenz', 'veranstaltung',
        'wochenvorschau', 'jahrespressekonferenz', 'pressegespräch', 'rundgang',
        'beginnt am', 'laden ein', 'lädt ein', 'sitzung vom'
    ]
    
    # 2. Regex-Muster für Datumsangaben in der Zukunft (relativ zum Erstelldatum)
    date_patterns = [
        r'am\s+\d{1,2}\.\s*\d{1,2}\.\s*\d{4}',  # "am 15.05.2023"
        r'vom\s+\d{1,2}\.\s*\d{1,2}\.',         # "vom 15.05."
        r'terminiert auf den \d{1,2}',           # "terminiert auf den 15"
        r'in der kommenden woche',               # zukünftige Verweise
        r'in der nächsten woche',
        r'demnächst'
    ]
    
    # 3. Überschriften/Anfänge, die auf Ankündigungen hindeuten
    headline_indicators = [
        'pressemitteilung nr', 'presseinformation', 'information für die presse',
        'medieninformation', 'zur information', 'mündliche verhandlung',
        'terminvorschau', 'jahresbericht', 'geschäftsbericht', 'tätigkeitsbericht',
        'stellenausschreibung', 'personelle veränderungen', 'neuer präsident'
    ]
    
    # 4. Prüfen auf Keywords am Anfang des Textes (größeres Gewicht)
    summary_start = summary[:100]
    headline_match = any(indicator in summary_start for indicator in headline_indicators)
    
    # 5. Prüfen auf allgemeine Ankündigungskeywords im gesamten Text
    keyword_match = any(re.search(r'\b' + re.escape(indicator) + r'\b', summary, re.IGNORECASE) 
                      for indicator in future_indicators)
    
    # 6. Prüfen auf Datumsmuster
    date_match = any(re.search(pattern, summary, re.IGNORECASE) for pattern in date_patterns)
    
    # Kombinierte Entscheidung mit unterschiedlicher Gewichtung
    if headline_match:
        return True  # Überschriften/Anfänge sind starke Indikatoren
    elif keyword_match and date_match:
        return True  # Kombination aus Keyword und Datumsmuster
    elif keyword_match and ('mündliche verhandlung' in summary or 'termin' in summary):
        return True  # Spezifische Kombinationen
    
    return False  # Default: keine Ankündigung

# Anwenden der regelbasierten Methode auf den Datensatz
df['is_announcement_rule'] = df.apply(is_announcement_rule_based, axis=1)

# Auswertung der regelbasierten Methode
announcement_count = df['is_announcement_rule'].sum()
print(f"Regelbasierte Erkennung: {announcement_count} von {len(df)} Einträgen ({announcement_count/len(df)*100:.2f}%) sind vermutlich Ankündigungen.")

# Beispiele für erkannte Ankündigungen anzeigen
print("\nBeispiele für erkannte Ankündigungen:")
announcement_examples = df[df['is_announcement_rule']].head(3)
for _, row in announcement_examples.iterrows():
    print(f"ID: {row['id']}")
    print(f"Datum: {row['date']}")
    print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
    print("-" * 80)

Regelbasierte Erkennung: 1625 von 6592 Einträgen (24.65%) sind vermutlich Ankündigungen.

Beispiele für erkannte Ankündigungen:
ID: bfh_009-17
Datum: 08. Februar 2017
Pressemitteilung (Anfang): Kostümparty eines gemeinnützigen Karnevalsvereins kein Zweckbetrieb
                
                  08. Februar 2017
                
                -
                  Nummer
                  00...
--------------------------------------------------------------------------------
ID: bfh_011-13
Datum: 20. Februar 2013
Pressemitteilung (Anfang): Kosten einer Betriebsveranstaltung sind erst bei Überschreiten einer Freigrenze Arbeitslohn. Die Freigrenze beträgt auch 2007 noch 110 €.
                
                  20. Februar 2013
          ...
--------------------------------------------------------------------------------
ID: bfh_032-17
Datum: 17. Mai 2017
Pressemitteilung (Anfang): Gewerbesteuerliche Hinzurechnung von Mieten für Konzertsäle
                
                  17. Mai 2017


## 3. Semantische Ähnlichkeit

Der zweite Ansatz nutzt TF-IDF Vektorisierung und Kosinus-Ähnlichkeit, um inhaltliche Unterschiede zwischen Urteilen und Pressemitteilungen zu erkennen:

- **Vorverarbeitung**: Lemmatisierung und Filterung mit spaCy
- **TF-IDF**: Berechnung von Dokumentvektoren
- **Ähnlichkeitsmetrik**: Kosinus-Ähnlichkeit zwischen Urteil und Pressemitteilung
- **Schwellenwert**: Identifikation von Paaren mit geringer Ähnlichkeit

Dieser Ansatz ist besonders effektiv bei der Erkennung von Pressemitteilungen, die inhaltlich stark vom zugehörigen Urteil abweichen.

In [4]:
### 2. Semantische Ähnlichkeit mit Embeddings

# Bei diesem Ansatz berechnen wir die semantische Ähnlichkeit zwischen Urteil und Pressemitteilung
# Pressemitteilungen mit niedriger Ähnlichkeit zum zugehörigen Urteil werden als potenziell irrelevant eingestuft

# Wir beginnen mit einem einfachen TF-IDF-basierten Ansatz
print("\n## Semantische Ähnlichkeit mit TF-IDF Embeddings")

# TF-IDF Vektorisierung für alle Texte
print("Berechne TF-IDF Vektoren...")

if USE_GPU:
    tfidf_vectorizer = cuTfidfVectorizer(
        max_features=10000,
        min_df=5,
        max_df=0.8,
        ngram_range=(1, 2),  # Uni- und Bi-gramme
        sublinear_tf=True
    )
else:
    tfidf_vectorizer = TfidfVectorizer(
        max_features=10000,
        min_df=5,
        max_df=0.8,
        ngram_range=(1, 2),  # Uni- und Bi-gramme
        sublinear_tf=True
    )

# Preprocessed Texte erstellen
print("Führe Text-Preprocessing durch...")
df['preprocessed_summary'] = df['summary'].fillna('').apply(preprocess_text)
df['preprocessed_judgement'] = df['judgement'].fillna('').apply(preprocess_text)

# Kombiniere alle Texte für das Training des Vektorisierers
all_texts = list(df['preprocessed_summary']) + list(df['preprocessed_judgement'])
tfidf_vectorizer.fit(all_texts)
print(f"Vokabulargröße: {len(tfidf_vectorizer.get_feature_names_out())}")

# Vektorisiere Urteile und Pressemitteilungen
summary_vectors = tfidf_vectorizer.transform(df['preprocessed_summary'])
judgement_vectors = tfidf_vectorizer.transform(df['preprocessed_judgement'])

# Berechne die Kosinus-Ähnlichkeit für jedes Paar
print("Berechne Kosinus-Ähnlichkeiten...")
similarities = []

# GPU-beschleunigte oder CPU-basierte Berechnung je nach Verfügbarkeit
if USE_GPU:
    for i in tqdm(range(len(df))):
        if i < summary_vectors.shape[0] and i < judgement_vectors.shape[0]:
            # Für GPU: normalisierte Vektoren und paarweise Ähnlichkeiten mit cuML
            summary_vec = cp.sparse.csr_matrix(summary_vectors[i])
            judgement_vec = cp.sparse.csr_matrix(judgement_vectors[i])
            
            # Umwandlung in dichte Arrays (notwendig für einige cuML-Operationen)
            summary_dense = summary_vec.toarray().astype(cp.float32)
            judgement_dense = judgement_vec.toarray().astype(cp.float32)
            
            # Normalisierung (für Kosinus-Ähnlichkeit)
            if cp.sum(summary_dense) > 0:
                summary_dense = normalize(summary_dense)
            if cp.sum(judgement_dense) > 0:
                judgement_dense = normalize(judgement_dense)
            
            # Kosinus-Ähnlichkeit = Skalarprodukt der normalisierten Vektoren
            similarity = cp.dot(summary_dense.flatten(), judgement_dense.flatten())
            similarities.append(float(similarity))
        else:
            similarities.append(0.0)
else:
    # Originale CPU-Implementierung
    for i in tqdm(range(len(df))):
        if i < summary_vectors.shape[0] and i < judgement_vectors.shape[0]:
            # Extrahiere die Vektoren für das aktuelle Paar
            summary_vec = summary_vectors[i]
            judgement_vec = judgement_vectors[i]
            
            # Berechne die Kosinus-Ähnlichkeit
            similarity = cosine_similarity(summary_vec, judgement_vec)[0][0]
            similarities.append(similarity)
        else:
            similarities.append(0.0)

# Speichere die Ähnlichkeiten im DataFrame
df['tfidf_similarity'] = similarities

# Analysiere die Ähnlichkeitsverteilung
plt.figure(figsize=(10, 6))
plt.hist(df['tfidf_similarity'], bins=50)
plt.title('Verteilung der TF-IDF Kosinus-Ähnlichkeiten')
plt.xlabel('Kosinus-Ähnlichkeit')
plt.ylabel('Anzahl der Paare')
plt.axvline(x=df['tfidf_similarity'].quantile(0.1), color='r', linestyle='--', 
            label=f'10% Quantil: {df["tfidf_similarity"].quantile(0.1):.3f}')
plt.legend()
plt.show()

# Definiere einen Schwellenwert für die Ähnlichkeit
# Wir nehmen an, dass die 10% der Paare mit der geringsten Ähnlichkeit 
# potenziell irrelevante Pressemitteilungen enthalten
similarity_threshold = df['tfidf_similarity'].quantile(0.1)
print(f"Schwellenwert für TF-IDF Ähnlichkeit: {similarity_threshold:.3f}")

# Identifiziere Paare mit niedriger Ähnlichkeit
df['is_dissimilar_tfidf'] = df['tfidf_similarity'] < similarity_threshold

# Auswertung der semantischen Ähnlichkeitsmethode
dissimilar_count = df['is_dissimilar_tfidf'].sum()
print(f"TF-IDF Ähnlichkeitserkennung: {dissimilar_count} von {len(df)} Einträgen ({dissimilar_count/len(df)*100:.2f}%) haben geringe Ähnlichkeit.")

# Vergleich mit regelbasierter Methode
overlap_count = (df['is_announcement_rule'] & df['is_dissimilar_tfidf']).sum()
print(f"Überlappung mit regelbasierter Methode: {overlap_count} Einträge ({overlap_count/dissimilar_count*100:.2f}% der Einträge mit geringer Ähnlichkeit)")

# Beispiele für Einträge mit niedriger Ähnlichkeit anzeigen
print("\nBeispiele für Einträge mit niedriger semantischer Ähnlichkeit:")
dissimilar_examples = df[df['is_dissimilar_tfidf']].sort_values('tfidf_similarity').head(3)
for _, row in dissimilar_examples.iterrows():
    print(f"ID: {row['id']}")
    print(f"Ähnlichkeit: {row['tfidf_similarity']:.4f}")
    print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
    print(f"Urteil (Anfang): {row['judgement'][:200]}...")
    print("-" * 80)


## Semantische Ähnlichkeit mit TF-IDF Embeddings
Berechne TF-IDF Vektoren...
Führe Text-Preprocessing durch...


KeyboardInterrupt: 

## 4. Überwachtes Machine Learning

Der dritte Ansatz nutzt die Ergebnisse der vorherigen Methoden als "schwache Labels" für einen ML-Klassifikator:

- **Feature-Engineering**: TF-IDF Vektorisierung der Texte
- **Modell**: Logistische Regression mit Klassengewichtung
- **Evaluation**: Confusion Matrix und Klassifikationsreport
- **Feature-Analyse**: Wichtigste Wörter für beide Klassen

Das trainierte Modell lernt komplexere Muster als die regelbasierten Ansätze und kann diese auf neue Texte anwenden.

In [ ]:
### 3. Überwachtes Machine Learning (Klassifikation)

# In diesem Ansatz trainieren wir ein Modell, das auf Basis von Texteigenschaften 
# automatisch zwischen relevanten und irrelevanten Pressemitteilungen unterscheidet

print("\n## Überwachtes Machine Learning (Klassifikation)")

# Da wir keine manuell gelabelten Daten haben, nutzen wir die Ergebnisse
# der vorherigen Methoden als "schwache" Labels für das Training
print("Erstelle Labels für das Trainieren basierend auf regelbasierter Methode...")

# Wir betrachten Einträge als "irrelevant" (Label 1), wenn sie entweder
# durch die regelbasierte Methode als Ankündigungen erkannt wurden
# oder eine sehr niedrige semantische Ähnlichkeit aufweisen
df['is_irrelevant'] = ((df['is_announcement_rule']) | 
                       (df['tfidf_similarity'] < similarity_threshold * 0.8))

# Balancieren der Daten (optional)
irrelevant_count = df['is_irrelevant'].sum()
print(f"Anzahl der als irrelevant markierten Einträge: {irrelevant_count} ({irrelevant_count/len(df)*100:.2f}%)")

# Features für das Training erstellen (TF-IDF auf den Pressemitteilungen)
print("Erstelle Features für das Klassifikationsmodell...")
classifier_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=3,
    max_df=0.85,
    ngram_range=(1, 2)
)

# Wir nutzen die Originaltexte (nicht die vorverarbeiteten)
# um auch strukturelle Merkmale zu erfassen
X = classifier_vectorizer.fit_transform(df['summary'].fillna(''))
y = df['is_irrelevant'].astype(int)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Trainingsdaten: {X_train.shape[0]} Einträge, Testdaten: {X_test.shape[0]} Einträge")
print(f"Klassen-Verteilung im Training: {Counter(y_train)}")

# Logistische Regression für Klassifikation
print("Trainiere Logistic Regression Klassifikator...")

if USE_GPU:
    # GPU-beschleunigte Logistische Regression
    classifier = cuLogisticRegression(
        C=1.0,
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    )
else:
    # CPU-basierte Logistische Regression
    classifier = LogisticRegression(
        C=1.0,
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    )

classifier.fit(X_train, y_train)

# Modell evaluieren
y_pred = classifier.predict(X_test)
y_pred_prob = classifier.predict_proba(X_test)[:, 1]

# Zeige Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Klassifikationsreport zeigen
print("\nKlassifikationsreport:")
print(classification_report(y_test, y_pred))

# Die wichtigsten Merkmale für jede Klasse anzeigen
print("\nWichtigste Merkmale für die Erkennung von irrelevanten Pressemitteilungen:")
if hasattr(classifier, 'coef_'):
    feature_names = classifier_vectorizer.get_feature_names_out()
    # Sortiere nach absoluten Koeffizienten (größte zuerst)
    sorted_coef_indices = np.argsort(np.abs(classifier.coef_[0]))[::-1]
    
    # Top N positive und negative Features
    n_features = 20
    
    # Features, die für "irrelevant" sprechen (positive Koeffizienten)
    print("Features, die für 'irrelevant' sprechen:")
    pos_indices = [idx for idx in sorted_coef_indices if classifier.coef_[0][idx] > 0][:n_features]
    for idx in pos_indices:
        print(f"  {feature_names[idx]}: {classifier.coef_[0][idx]:.4f}")
    
    # Features, die für "relevant" sprechen (negative Koeffizienten)
    print("\nFeatures, die für 'relevant' sprechen:")
    neg_indices = [idx for idx in sorted_coef_indices if classifier.coef_[0][idx] < 0][:n_features]
    for idx in neg_indices:
        print(f"  {feature_names[idx]}: {classifier.coef_[0][idx]:.4f}")

# Anwenden des Modells auf den gesamten Datensatz
print("\nWende trainiertes Modell auf den gesamten Datensatz an...")
df['irrelevant_prob'] = classifier.predict_proba(X)[:, 1]
df['is_irrelevant_ml'] = classifier.predict(X)

# Auswertung der ML-Methode
ml_irrelevant_count = df['is_irrelevant_ml'].sum()
print(f"ML-Klassifikator: {ml_irrelevant_count} von {len(df)} Einträgen ({ml_irrelevant_count/len(df)*100:.2f}%) wurden als irrelevant klassifiziert.")

# Vergleich mit vorherigen Methoden
print("\nÜberlappung mit anderen Methoden:")
print(f"- Mit regelbasierter Methode: {(df['is_announcement_rule'] & df['is_irrelevant_ml']).sum()} Einträge")
print(f"- Mit TF-IDF Ähnlichkeit: {(df['is_dissimilar_tfidf'] & df['is_irrelevant_ml']).sum()} Einträge")

# Visualisierung der Wahrscheinlichkeiten
plt.figure(figsize=(10, 6))
sns.histplot(df['irrelevant_prob'], bins=50, kde=True)
plt.title('Verteilung der ML-Wahrscheinlichkeiten für "irrelevant"')
plt.xlabel('Wahrscheinlichkeit')
plt.ylabel('Anzahl der Einträge')
plt.axvline(x=0.5, color='red', linestyle='--')
plt.grid(True, alpha=0.3)
plt.show()

# Beispiele für als irrelevant klassifizierte Einträge anzeigen
print("\nBeispiele für als irrelevant klassifizierte Einträge:")
high_confidence_irrelevant = df[df['is_irrelevant_ml'] & (df['irrelevant_prob'] > 0.9)].sort_values('irrelevant_prob', ascending=False).head(3)
for _, row in high_confidence_irrelevant.iterrows():
    print(f"ID: {row['id']}")
    print(f"Irrelevanz-Wahrscheinlichkeit: {row['irrelevant_prob']:.4f}")
    print(f"Regelbasiert: {'Ja' if row['is_announcement_rule'] else 'Nein'}, TF-IDF unähnlich: {'Ja' if row['is_dissimilar_tfidf'] else 'Nein'}")
    print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
    print("-" * 80)

## 5. Unüberwachtes Clustering

Der vierte Ansatz gruppiert die Pressemitteilungen basierend auf ihrer inhaltlichen Ähnlichkeit:

- **Dimensionsreduktion**: PCA für effizienteres Clustering
- **K-Means**: Clustering in verschiedene Themenbereiche
- **Cluster-Analyse**: Identifikation von Ankündigungs-Clustern
- **Validierung**: Vergleich mit anderen Methoden

Diese Methode hilft dabei, natürliche Gruppierungen in den Daten zu finden und potenzielle Ankündigungscluster zu identifizieren.

In [ ]:
### 4. Unüberwachte Verfahren (Clustering/Topic Modeling)

# In diesem Ansatz gruppieren wir die Pressemitteilungen anhand ihrer Ähnlichkeit,
# um Cluster zu identifizieren, die potenziell Ankündigungen oder irrelevante Inhalte enthalten

print("\n## Unüberwachte Verfahren (Clustering/Topic Modeling)")

# Wir nutzen die bereits berechneten TF-IDF Vektoren der Pressemitteilungen
print("Verwende TF-IDF Vektoren für Clustering...")

# Dimensionsreduktion mit PCA und K-Means Clustering
print("Reduziere Dimensionalität mit PCA und führe Clustering durch...")

if USE_GPU:
    # GPU-beschleunigte PCA
    pca = cuPCA(n_components=50, copy=True)
    summary_vectors_dense = summary_vectors.toarray().astype(cp.float32)
    summary_vectors_reduced = pca.fit_transform(summary_vectors_dense)
    
    # GPU-beschleunigtes K-Means
    n_clusters = 5  # Wir probieren 5 Cluster
    kmeans = cuKMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(summary_vectors_reduced)
else:
    # CPU-basierte PCA
    pca = PCA(n_components=50)  # Wir reduzieren auf 50 Dimensionen
    summary_vectors_dense = summary_vectors.toarray()
    summary_vectors_reduced = pca.fit_transform(summary_vectors_dense)
    
    # CPU-basiertes K-Means
    n_clusters = 5  # Wir probieren 5 Cluster
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(summary_vectors_reduced)

# Speichere die Cluster-Labels im DataFrame
df['cluster'] = cluster_labels

print(f"Erklärte Varianz durch PCA: {sum(pca.explained_variance_ratio_):.2f}")

# Visualisierung der Cluster (auf 2 Dimensionen reduziert für die Darstellung)
print("Reduziere auf 2 Dimensionen für Visualisierung...")
pca_vis = PCA(n_components=2)
summary_vectors_2d = pca_vis.fit_transform(summary_vectors_reduced)

# Plotte die Cluster
plt.figure(figsize=(12, 10))
scatter = plt.scatter(summary_vectors_2d[:, 0], summary_vectors_2d[:, 1], 
                      c=cluster_labels, cmap='viridis', alpha=0.7, s=30)
plt.colorbar(scatter, label='Cluster')
plt.title('K-Means Clustering der Pressemitteilungen (PCA-reduziert)')
plt.xlabel('PCA Komponente 1')
plt.ylabel('PCA Komponente 2')
plt.grid(True, alpha=0.3)
plt.show()

# Analysiere die Cluster: Welche Cluster enthalten mehr regelbasiert erkannte Ankündigungen?
print("\nAnalyse der Cluster im Vergleich zu regelbasierten Erkennungen:")
cluster_analysis = df.groupby('cluster')['is_announcement_rule'].mean().sort_values(ascending=False)
print("Anteil der Ankündigungen in jedem Cluster (regelbasiert):")
print(cluster_analysis)

# Bestimme den Cluster mit dem höchsten Anteil an Ankündigungen
announcement_cluster = cluster_analysis.index[0]
print(f"\nCluster {announcement_cluster} hat den höchsten Anteil an potenziellen Ankündigungen ({cluster_analysis[0]:.2%})")

# Extrahiere die wichtigsten Wörter für jeden Cluster
print("\nWichtigste Wörter pro Cluster:")
# Verwende den TF-IDF Vektorisierer, um die wichtigsten Wörter zu identifizieren
feature_names = tfidf_vectorizer.get_feature_names_out()

# Für jeden Cluster die Top-Wörter basierend auf den Cluster-Zentroiden bestimmen
for i in range(n_clusters):
    # Hole die Indizes der Dokumente in diesem Cluster
    cluster_docs = df[df['cluster'] == i].index
    
    if len(cluster_docs) > 0:
        # Berechne die durchschnittlichen TF-IDF-Werte für diesen Cluster
        cluster_tfidf = summary_vectors[cluster_docs].mean(axis=0)
        cluster_tfidf = np.asarray(cluster_tfidf).flatten()
        
        # Sortiere nach TF-IDF-Werten und hole die Top-Wörter
        top_indices = np.argsort(cluster_tfidf)[::-1][:20]
        top_words = [feature_names[idx] for idx in top_indices]
        
        # Zeige die Top-Wörter an
        print(f"Cluster {i} (n={len(cluster_docs)}) - Top-Wörter: {', '.join(top_words[:10])}")
        
        # Einige Beispiele aus dem Cluster anzeigen
        if i == announcement_cluster:
            print("\nBeispiele aus dem erkannten 'Ankündigungs'-Cluster:")
            cluster_examples = df[df['cluster'] == i].sample(min(3, len(df[df['cluster'] == i]))).reset_index(drop=True)
            for idx, row in cluster_examples.iterrows():
                print(f"Beispiel {idx+1}:")
                print(f"ID: {row['id']}")
                print(f"Regelbasiert als Ankündigung erkannt: {'Ja' if row['is_announcement_rule'] else 'Nein'}")
                print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
                print("-" * 80)

# Vergleiche Cluster-basierte Erkennung mit anderen Methoden
# Wir betrachten den identifizierten Ankündigungs-Cluster als potenziell irrelevant
df['is_irrelevant_cluster'] = df['cluster'] == announcement_cluster

# Auswertung der Cluster-Methode
cluster_irrelevant_count = df['is_irrelevant_cluster'].sum()
print(f"\nCluster-Methode: {cluster_irrelevant_count} von {len(df)} Einträgen ({cluster_irrelevant_count/len(df)*100:.2f}%) im Ankündigungs-Cluster")

# Überlappung mit anderen Methoden
print("\nÜberlappung mit anderen Methoden:")
print(f"- Mit regelbasierter Methode: {(df['is_announcement_rule'] & df['is_irrelevant_cluster']).sum()} Einträge")
print(f"- Mit TF-IDF Ähnlichkeit: {(df['is_dissimilar_tfidf'] & df['is_irrelevant_cluster']).sum()} Einträge")
print(f"- Mit ML-Klassifikator: {(df['is_irrelevant_ml'] & df['is_irrelevant_cluster']).sum()} Einträge")

## 6. Vergleich und Kombination der Methoden

Im letzten Abschnitt vergleichen wir die Ergebnisse aller Methoden und kombinieren sie zu einem robusten Gesamtansatz:

- **Methodenvergleich**: Überlappung und Unterschiede
- **Ensemble-Ansatz**: Kombination durch Mehrheitsentscheidung
- **Grenzfälle**: Analyse von Beispielen mit unterschiedlichen Klassifikationen
- **Finaler Datensatz**: Bereinigung basierend auf kombiniertem Ansatz

Der finale bereinigte Datensatz enthält nur noch Pressemitteilungen, die von der Mehrheit der Methoden als relevant eingestuft wurden.

In [ ]:
## Zusammenfassung und Vergleich der Methoden

# Nach der Implementierung aller vier Ansätze vergleichen wir nun die Ergebnisse

print("\n## Vergleich der verschiedenen Bereinigungsansätze")

# Erstelle einen Vergleich der verschiedenen Methoden
comparison_df = pd.DataFrame({
    'Methode': [
        '1. Regelbasiert (Keywords/Patterns)',
        '2. Semantische Ähnlichkeit (TF-IDF)',
        '3. ML-Klassifikation',
        '4. Clustering'
    ],
    'Anzahl irrelevanter Einträge': [
        df['is_announcement_rule'].sum(),
        df['is_dissimilar_tfidf'].sum(),
        df['is_irrelevant_ml'].sum(),
        df['is_irrelevant_cluster'].sum()
    ]
})

# Berechne den Prozentsatz
comparison_df['Prozent vom Datensatz'] = comparison_df['Anzahl irrelevanter Einträge'] / len(df) * 100

# Zeige Vergleichstabelle
print(comparison_df.to_string(index=False))

# Überlappung der Methoden visualisieren
plt.figure(figsize=(12, 10))

# Venn-Diagramm Daten vorbereiten
from matplotlib_venn import venn3, venn3_circles

# Erstelle Sets der irrelevanten Einträge für jede Methode
rule_based_set = set(df[df['is_announcement_rule']].index)
similarity_set = set(df[df['is_dissimilar_tfidf']].index)
ml_set = set(df[df['is_irrelevant_ml']].index)

# Venn-Diagramm zeichnen
venn = venn3(
    [rule_based_set, similarity_set, ml_set],
    ('Regelbasiert', 'Semantische Ähnlichkeit', 'ML-Klassifikation')
)
plt.title('Überlappung der erkannten irrelevanten Einträge')
plt.show()

# Kombinierter Ansatz: Einträge identifizieren, die von mindestens 2 Methoden als irrelevant eingestuft wurden
print("\n## Kombinierter Bereinigungsansatz")

# Zähle, von wie vielen Methoden ein Eintrag als irrelevant eingestuft wurde
df['irrelevant_votes'] = (
    df['is_announcement_rule'].astype(int) +
    df['is_dissimilar_tfidf'].astype(int) +
    df['is_irrelevant_ml'].astype(int) +
    df['is_irrelevant_cluster'].astype(int)
)

# Analysiere die Verteilung der Stimmen
vote_counts = df['irrelevant_votes'].value_counts().sort_index()
print("Verteilung der 'irrelevant'-Stimmen:")
for votes, count in vote_counts.items():
    print(f"{votes} Methode(n): {count} Einträge ({count/len(df)*100:.2f}%)")

# Wir betrachten Einträge als irrelevant, wenn mindestens 2 Methoden sie als irrelevant eingestuft haben
vote_threshold = 2
df['is_irrelevant_combined'] = df['irrelevant_votes'] >= vote_threshold

# Auswertung des kombinierten Ansatzes
combined_irrelevant_count = df['is_irrelevant_combined'].sum()
print(f"\nKombinierter Ansatz (≥{vote_threshold} Methoden): {combined_irrelevant_count} von {len(df)} Einträgen ({combined_irrelevant_count/len(df)*100:.2f}%) als irrelevant identifiziert.")

# Beispiele für Einträge, die von allen Methoden als irrelevant eingestuft wurden
full_agreement = df[df['irrelevant_votes'] == 4].head(3)
print("\nBeispiele für Einträge, die von ALLEN Methoden als irrelevant eingestuft wurden:")
for _, row in full_agreement.iterrows():
    print(f"ID: {row['id']}")
    print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
    print("-" * 80)

# Beispiele für Grenzfälle (genau am Schwellenwert)
borderline_cases = df[df['irrelevant_votes'] == vote_threshold].head(3)
print(f"\nBeispiele für Grenzfälle (genau {vote_threshold} Methoden):")
for _, row in borderline_cases.iterrows():
    print(f"ID: {row['id']}")
    print(f"Regelbasiert: {'Ja' if row['is_announcement_rule'] else 'Nein'}, "
          f"TF-IDF unähnlich: {'Ja' if row['is_dissimilar_tfidf'] else 'Nein'}, "
          f"ML: {'Ja' if row['is_irrelevant_ml'] else 'Nein'}, "
          f"Cluster: {'Ja' if row['is_irrelevant_cluster'] else 'Nein'}")
    print(f"Pressemitteilung (Anfang): {row['summary'][:200]}...")
    print("-" * 80)

## Anwendung der kombinierten Methode auf den Datensatz

# Bereinigten Datensatz erstellen
cleaned_df = df[~df['is_irrelevant_combined']].copy()
print(f"\nBereinigter Datensatz: {len(cleaned_df)} Einträge (von ursprünglich {len(df)})")
print(f"Entfernt: {len(df) - len(cleaned_df)} Einträge ({(len(df) - len(cleaned_df))/len(df)*100:.2f}%)")

# Speichern des bereinigten Datensatzes
output_dir = Path('cleaned_data')
output_dir.mkdir(exist_ok=True)

# Speichere den bereinigten Datensatz
cleaned_output_file = output_dir / "cleaned_combined_methods.csv"
cleaned_df.to_csv(cleaned_output_file, index=False)
print(f"Bereinigter Datensatz gespeichert unter: {cleaned_output_file}")

# Speichere auch die Metadaten der Bereinigung
metadata_df = df[['id', 'is_announcement_rule', 'tfidf_similarity', 'is_dissimilar_tfidf', 
                 'is_irrelevant_ml', 'irrelevant_prob', 'cluster', 'is_irrelevant_cluster',
                 'irrelevant_votes', 'is_irrelevant_combined']]
metadata_output_file = output_dir / "cleaning_metadata.csv"
metadata_df.to_csv(metadata_output_file, index=False)
print(f"Bereinigungsmetadaten gespeichert unter: {metadata_output_file}")

print("\nBereinigungsprozess abgeschlossen!")